# Домашняя работа

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("ds_salaries.csv")

In [3]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [4]:
df = pd.get_dummies(df, drop_first= True )

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

In [6]:
X_train.shape, X_test.shape, X_val.shape

((2628, 269), (563, 269), (564, 269))

In [7]:
X_train = np.array(X_train)
y_train = np.array(y_train)

## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('mean absolute percentage error: ',mean_absolute_percentage_error(y_val, linreg.predict(np.array(X_val))))
print('mean squared error: ', np.sqrt(mean_squared_error(y_val, linreg.predict(np.array(X_val)))))

mean absolute percentage error:  0.4421670476813525
mean squared error:  51972.35883767933


## Задание 3 (0.5 балла)

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [9]:
!pip install catboost;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [11]:
cat.fit(X_train,y_train, verbose = False)

In [12]:
print('mean absolute percentage error: ',mean_absolute_percentage_error(y_val, cat.predict(X_val)))
print('mean squared error: ', np.sqrt(mean_squared_error(y_val, cat.predict(X_val))))

mean absolute percentage error:  0.4039858476909781
mean squared error:  49846.52187501451


## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [13]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [14]:
df = pd.read_csv("ds_salaries.csv")

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

In [15]:
onehot_encoder=OneHotEncoder(handle_unknown='ignore')
X_train_onehot=onehot_encoder.fit_transform(X_train)
X_val_onehot=onehot_encoder.transform(X_val)


linreg_onehot = LinearRegression()
linreg_onehot.fit(X_train_onehot, y_train)

print('mean absolute percentage error for linreg: ',mean_absolute_percentage_error(y_val, linreg_onehot.predict(X_val_onehot)))
print('mean squared error for linreg: ', np.sqrt(mean_squared_error(y_val, linreg_onehot.predict(X_val_onehot))))


cat_onehot=CatBoostRegressor()
cat_onehot.fit(X_train_onehot,y_train, verbose = False)

print()
print('mean absolute percentage error for boosting: ',mean_absolute_percentage_error(y_val, cat_onehot.predict(X_val_onehot)))
print('mean squared error for boosting: ', np.sqrt(mean_squared_error(y_val, cat_onehot.predict(X_val_onehot))))

mean absolute percentage error for linreg:  0.4430596384298492
mean squared error for linreg:  51787.75930397386

mean absolute percentage error for boosting:  0.4036771661004771
mean squared error for boosting:  50140.56189474383


In [16]:
label_encoder = LabelEncoder()
X_train_label = X_train.copy()
X_val_label = X_val.copy()

for col in X_train.columns.to_list():
    labels = X_train_label[col].unique().tolist()+['unknown']
    label_encoder.fit(labels)
    X_val_label[col] = X_val_label[col].map(lambda s:'unknown' if s not in label_encoder.classes_ else s)
    X_train_label[col] = label_encoder.transform(X_train_label[col])
    X_val_label[col] = label_encoder.transform(X_val_label[col])



linreg_label = LinearRegression()
linreg_label.fit(X_train_label, y_train)

print('mean absolute percentage error for linreg: ',mean_absolute_percentage_error(y_val, linreg_label.predict(X_val_label)))
print('mean squared error for linreg: ', np.sqrt(mean_squared_error(y_val, linreg_label.predict(X_val_label))))

cat_label=CatBoostRegressor()
cat_label.fit(X_train_label,y_train, verbose = False)

print()
print('mean absolute percentage error for boosting: ',mean_absolute_percentage_error(y_val, cat_label.predict(X_val_label)))
print('mean squared error for boosting: ', np.sqrt(mean_squared_error(y_val, cat_label.predict(X_val_label))))

mean absolute percentage error for linreg:  0.6514078991842985
mean squared error for linreg:  58399.168505486996


<ipython-input-16-0f6abbed305a>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  X_val_label[col] = X_val_label[col].map(lambda s:'unknown' if s not in label_encoder.classes_ else s)
<ipython-input-16-0f6abbed305a>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  X_val_label[col] = X_val_label[col].map(lambda s:'unknown' if s not in label_encoder.classes_ else s)



mean absolute percentage error for boosting:  0.5016971131445227
mean squared error for boosting:  53525.068660399804


In [17]:
X_train_label.head(10)

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
1661,3,2,2,32,16,65,0,64,1
3414,2,3,2,45,16,65,1,64,1
1829,2,3,2,40,16,65,0,64,1
2233,2,3,2,30,16,65,1,64,1
2380,2,0,2,66,16,36,1,19,2
169,3,3,2,4,16,65,1,64,1
2905,2,3,2,45,16,65,1,64,1
2431,2,3,2,32,16,65,0,64,1
2768,2,3,2,45,16,65,1,64,1
3281,2,3,2,45,16,65,1,64,1


In [18]:
X_train_target = X_train_label.copy()
X_val_target = X_val_label.copy()
X_train_target['salary_in_usd'] = y_train

for col in X_train.columns.tolist():
  mean_target = X_train_target.groupby(col)['salary_in_usd'].mean()
  X_train_target[col] = X_train_target[col].map(mean_target)

X_train_target = X_train_target.drop(['salary_in_usd'], axis = 1)

linreg_target = LinearRegression()
linreg_target.fit(X_train_target, y_train)

print('mean absolute percentage error for linreg: ',mean_absolute_percentage_error(y_val, linreg_target.predict(X_val_target)))
print('mean squared error for linreg: ', np.sqrt(mean_squared_error(y_val, linreg_target.predict(X_val_target))))

cat_target=CatBoostRegressor()
cat_target.fit(X_train_target,y_train, verbose = False)

print()
print('mean absolute percentage error for boosting: ',mean_absolute_percentage_error(y_val, cat_target.predict(X_val_target)))
print('mean squared error for boosting: ', np.sqrt(mean_squared_error(y_val, cat_target.predict(X_val_target))))

mean absolute percentage error for linreg:  3.0223538535164503
mean squared error for linreg:  326074.81630707794

mean absolute percentage error for boosting:  0.8142837028564313
mean squared error for boosting:  133225.84292542015


## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [19]:
df = pd.read_csv("ds_salaries.csv")
df = pd.get_dummies(df, drop_first= True )

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

X_train = np.array(X_train)
y_train = np.array(y_train)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [20]:
errors = [[abs(np.array(y_val)[i] - linreg.predict(np.array([X_val.iloc[1]]))), i] for i in range(X_val.shape[0])]

In [21]:
# errors = [[abs(np.array(y_val)[i] - cat.predict(np.array([X_val.iloc[1]]))), i] for i in range(X_val.shape[0])]

In [22]:
errors.sort()

In [23]:
list_of_i = []
for i in range(20):
  list_of_i.append(errors[-20:][i][1])

new_list_of_i = X_val.reset_index()['index'].loc[list_of_i].tolist()

In [24]:
df = pd.read_csv("ds_salaries.csv")
X = df.drop(['salary_in_usd', 'salary'],axis = 1)

In [25]:
# То, что модель хуже всего предсказывает
X.loc[new_list_of_i]

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
1558,2023,SE,FT,Data Science Manager,USD,US,0,US,M
2670,2022,SE,FT,Data Engineer,USD,US,0,US,M
2856,2022,SE,FT,Data Engineer,USD,US,0,US,M
3703,2021,MI,FT,Data Scientist,BRL,BR,0,BR,S
1547,2023,EN,FT,Business Data Analyst,EUR,GR,50,GR,L
1341,2023,EN,FT,Data Scientist,INR,IN,50,IN,L
2427,2022,SE,FT,Cloud Data Engineer,EUR,SK,100,SK,S
2034,2022,EN,FT,Machine Learning Engineer,USD,AR,100,AR,L
3735,2021,EN,PT,AI Scientist,USD,BR,100,US,S
358,2023,SE,FT,Machine Learning Engineer,USD,US,100,US,M


In [26]:
# То, что модель лучше всего предсказывает
list_of_i = []
for i in range(20):
  list_of_i.append(errors[:20][i][1])

new_list_of_i = X_val.reset_index()['index'].loc[list_of_i].tolist()

X.loc[new_list_of_i]

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
3305,2022,SE,FT,Data Engineer,USD,US,100,US,M
2409,2022,SE,FT,Data Scientist,USD,US,100,US,M
1359,2023,SE,FT,Data Scientist,USD,US,100,US,M
1283,2023,SE,FT,Data Science Lead,USD,US,0,US,M
1544,2023,SE,FT,Data Scientist,USD,US,0,US,M
1385,2023,SE,FT,Data Scientist,USD,US,0,US,M
2248,2022,SE,FT,Data Engineer,USD,US,0,US,M
1469,2023,MI,FT,Data Analyst,USD,US,0,US,M
2801,2022,SE,FT,Data Engineer,USD,US,100,US,M
1218,2023,SE,FT,Data Architect,USD,US,100,US,M


Видимо, модель ошибается в случае, если компания не находится в US. В наилучших предсказываниях все люди работали в Омэрике, когда как в худших предсказываниях лишь 9 были из US

## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

In [27]:
# df = pd.read_csv("ds_salaries.csv")
# df = pd.get_dummies(df, drop_first= True )

# X = df.drop(['salary_in_usd', 'salary'],axis = 1)
# y = df['salary_in_usd']

In [28]:
df['in_US'] = (df['company_location'] == 'US').map(lambda x: 1 if x else 0)

df['in_US'] = df['in_US'] * df['remote_ratio'] / 100

In [29]:
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,in_US
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L,0.0
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S,1.0
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S,1.0
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M,0.0
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L,1.0
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L,1.0
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S,1.0
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L,1.0


In [30]:
df = pd.get_dummies(df, drop_first= True )

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('mean absolute percentage error: ',mean_absolute_percentage_error(y_val, linreg.predict(X_val)))
print('mean squared error: ', np.sqrt(mean_squared_error(y_val, linreg.predict(X_val))))

mean absolute percentage error:  0.4415562459272788
mean squared error:  51949.38760654537


Лучше не стало :(

## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

In [32]:
df = pd.read_csv("ds_salaries.csv")

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

In [37]:
from catboost import Pool, cv
import matplotlib.pyplot as plt

params = {"iterations": 10,
          "depth": 4,
          "loss_function": "RMSE",
          "verbose": False}
cv_dataset = Pool(data=X_train, label=y_train,
                  cat_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 'employee_residence', 'company_location', 'company_size'])
scores = cv(cv_dataset,
            params,
            fold_count=3,
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/3]

bestTest = 119405.5834
bestIteration = 9

Training on fold [1/3]

bestTest = 116130.328
bestIteration = 9

Training on fold [2/3]

bestTest = 119433.9381
bestIteration = 9



In [38]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(verbose=False)

In [39]:
cat_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 'employee_residence', 'company_location', 'company_size']

In [40]:
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        }


grid_search_result = cat.grid_search(grid, cv_dataset)
print(grid_search_result['params'])
print(grid_search_result['cv_results'])


bestTest = 48949.93553
bestIteration = 268

0:	loss: 48949.9355345	best: 48949.9355345 (0)	total: 3.03s	remaining: 1m 27s

bestTest = 49014.2322
bestIteration = 115

1:	loss: 49014.2322042	best: 48949.9355345 (0)	total: 5.97s	remaining: 1m 23s

bestTest = 48812.22268
bestIteration = 585

2:	loss: 48812.2226793	best: 48812.2226793 (2)	total: 10s	remaining: 1m 30s

bestTest = 48842.5126
bestIteration = 259

3:	loss: 48842.5126005	best: 48812.2226793 (2)	total: 13.4s	remaining: 1m 26s

bestTest = 48935.66073
bestIteration = 697

4:	loss: 48935.6607292	best: 48812.2226793 (2)	total: 17.1s	remaining: 1m 25s

bestTest = 48748.20823
bestIteration = 136

5:	loss: 48748.2082267	best: 48748.2082267 (5)	total: 20.7s	remaining: 1m 22s

bestTest = 48925.7312
bestIteration = 565

6:	loss: 48925.7312018	best: 48748.2082267 (5)	total: 25.5s	remaining: 1m 23s

bestTest = 48800.32231
bestIteration = 215

7:	loss: 48800.3223093	best: 48748.2082267 (5)	total: 28.3s	remaining: 1m 17s

bestTest = 48908.417

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [41]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import sklearn

In [43]:
df = pd.read_csv("ds_salaries.csv")
df = pd.get_dummies(df, drop_first= True )

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

In [44]:
model = XGBRegressor()

In [45]:
grid={"learning_rate": (0.03, 0.10),
                        "max_depth": [4, 6, 10],
                        "min_child_weight": [ 1, 3, 5, 7],
      }

In [46]:
grid_search = GridSearchCV(
    estimator=model,
    param_grid= grid,
    scoring = 'neg_mean_squared_error',
    cv = 4,
    n_jobs = -1,
    verbose = 0
)

In [47]:
X_train

,work_year,remote_ratio,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title_AI Developer,job_title_AI Programmer,...,company_location_SG,company_location_SI,company_location_SK,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_size_M,company_size_S
1661,2023,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3414,2022,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1829,2022,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2233,2022,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2380,2022,100,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,2023,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1785,2023,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1907,2022,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3216,2022,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [48]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=4,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': (0.03, 0.1), 'max_depth': [4, 6, 10],
                         'min_child_weight': [1, 3, 5, 7]},
             scoring='neg_mean_squared_error')

In [49]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 1}

In [50]:
model = XGBRegressor(learning_rate = 0.1, max_depth = 6, min_child_weight = 1)
model.fit(X_train, y_train)

print('mean absolute percentage error: ',mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('mean squared error: ', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

mean absolute percentage error:  0.4013125338047348
mean squared error:  49281.91256590011


## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [51]:
pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import lightgbm
from lightgbm import LGBMRegressor

In [53]:
model = LGBMRegressor()

In [54]:
grid={"learning_rate": (0.03, 0.10),
                        "max_depth": [4, 6, 10],
                        "min_child_weight": [ 1, 3, 5, 7],
      }

In [55]:
grid_search = GridSearchCV(
    estimator=model,
    param_grid= grid,
    scoring = 'neg_mean_squared_error',
    cv = 4,
    n_jobs = -1,
    verbose = 0
)

In [56]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': (0.03, 0.1), 'max_depth': [4, 6, 10],
                         'min_child_weight': [1, 3, 5, 7]},
             scoring='neg_mean_squared_error')

In [57]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 1}

In [58]:
model = LGBMRegressor(learning_rate = 0.1, max_depth = 6, min_child_weight = 1)
model.fit(X_train, y_train)

print('mean absolute percentage error: ',mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('mean squared error: ', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

mean absolute percentage error:  0.4063914470527593
mean squared error:  49807.38580508824


## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания

In [59]:
%%time
for i in range(10):
  model = LGBMRegressor(learning_rate = 0.1, max_depth = 6, min_child_weight = 1)
  model.fit(X_train, y_train)

CPU times: user 2.81 s, sys: 38.5 ms, total: 2.85 s
Wall time: 3.24 s


In [60]:
%%time
print('mean absolute percentage error: ',mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('mean squared error: ', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

mean absolute percentage error:  0.4063914470527593
mean squared error:  49807.38580508824
CPU times: user 57.1 ms, sys: 0 ns, total: 57.1 ms
Wall time: 68.6 ms


In [61]:
%%time
for i in range(10):
  model = XGBRegressor(learning_rate = 0.1, max_depth = 6, min_child_weight = 1)
  model.fit(X_train, y_train)

CPU times: user 1min 4s, sys: 143 ms, total: 1min 4s
Wall time: 46.8 s


In [62]:
%%time
print('mean absolute percentage error: ',mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('mean squared error: ', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

mean absolute percentage error:  0.4013125338047348
mean squared error:  49281.91256590011
CPU times: user 26.2 ms, sys: 1.01 ms, total: 27.2 ms
Wall time: 15.3 ms


In [63]:
%%time
for i in range(10):
  model = CatBoostRegressor(verbose=False, depth = 6, l2_leaf_reg = 3, learning_rate = 0.03)
  model.fit(X_train, y_train)

CPU times: user 24.6 s, sys: 2.98 s, total: 27.6 s
Wall time: 16.3 s


In [64]:
print('mean absolute percentage error: ',mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('mean squared error: ', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

mean absolute percentage error:  0.4063603308013601
mean squared error:  49704.2119864477


LGBMRegressor оказался самым быстрым в обучении (~1.07 сек), следом идёт Catbost (~20.5 сек) и заканчивает тройку XGB (~57 сек). Конечно, такие тесты не совсем корректны, у моделей были различные параметры, оптимальные только для них, однако такие тесты показывают они показывают хотя бы приближённую картину.

Что же касается точности, то тут все модели показали одинаковый результат